In [9]:
# Install dependencies as needed:
# !pip install --upgrade kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews",force_download=True)

print("Path to dataset files:", path)

100%|███████████████████████████████████████████████████████████████████████████████| 152M/152M [00:05<00:00, 28.3MB/s]

Extracting files...


Path to dataset files: C:\Users\suny5\.cache\kagglehub\datasets\andrezaza\clapper-massive-rotten-tomatoes-movies-and-reviews\versions\4


In [10]:
# check if the files are successfully downloaded in the said path
# on Mac
!ls -alh $path
# on Windows
!dir $path 

'ls' is not recognized as an internal or external command,
operable program or batch file.


 Volume in drive C is Windows
 Volume Serial Number is 7047-86E7

 Directory of C:\Users\suny5\.cache\kagglehub\datasets\andrezaza\clapper-massive-rotten-tomatoes-movies-and-reviews\versions\4

03/05/2025  11:26 AM    <DIR>          .
03/05/2025  11:08 AM    <DIR>          ..
03/05/2025  11:26 AM        17,396,636 rotten_tomatoes_movies.csv
03/05/2025  11:26 AM       410,691,413 rotten_tomatoes_movie_reviews.csv
               2 File(s)    428,088,049 bytes
               2 Dir(s)  419,652,173,824 bytes free


## Using the _movies.csv file that contains basic information and ratings for each movie.

### rotten_tomatoes_movies.csv - Each record represents a movie available on Rotten Tomatoes and includes all fields available on the website
*Note that 'rating' field here refers to the age-based rating (e.g. G) and not the rating by viewers.

- id - Unique identifier for each movie
- title - The title of the movie
- audienceScore - The average score given by regular viewers
- tomatoMeter - The percentage of positive reviews from professional critics
- rating - The movie's age-based classification (e.g., G, PG, PG-13, R)
- ratingContents - Content leading to the rating classification
- releaseDateTheaters - The date the movie was released in theaters
- releaseDateStreaming - The date the movie became available for streaming
- runtimeMinutes - The duration of the movie in minutes
- genre - The movie's genre(s)
- originalLanguage - The original language of the movie
- director - The movie's director
- writer - The writer(s) responsible for the movie's screenplay
- boxOffice - The movie's total box office revenue
- distributer - The company responsible for distributing the movie
- soundMix - The audio format(s) used in the movie

Load the two tsv files to dataframes and then write to a sqlite table.

In [5]:
import pandas as pd
import os
import sqlite3 

In [11]:
# Load CSV file into a DataFrame
df_rt = pd.read_csv(os.path.join(path,"rotten_tomatoes_movies.csv"))
df_rt = df_rt.convert_dtypes()
df_rt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143258 entries, 0 to 143257
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    143258 non-null  string
 1   title                 142891 non-null  string
 2   audienceScore         73248 non-null   Int64 
 3   tomatoMeter           33877 non-null   Int64 
 4   rating                13991 non-null   string
 5   ratingContents        13991 non-null   string
 6   releaseDateTheaters   30773 non-null   string
 7   releaseDateStreaming  79420 non-null   string
 8   runtimeMinutes        129431 non-null  Int64 
 9   genre                 132175 non-null  string
 10  originalLanguage      129400 non-null  string
 11  director              139041 non-null  string
 12  writer                90116 non-null   string
 13  boxOffice             14743 non-null   string
 14  distributor           23001 non-null   string
 15  soundMix         

In [12]:
df_rt.head()

,id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix
0,space-zombie-bingo,Space Zombie Bingo!,50,<NA>,<NA>,<NA>,<NA>,2018-08-25,75,"Comedy, Horror, Sci-fi",English,George Ormrod,"George Ormrod,John Sabotta",<NA>,<NA>,<NA>
1,the_green_grass,The Green Grass,<NA>,<NA>,<NA>,<NA>,<NA>,2020-02-11,114,Drama,English,Tiffany Edwards,Tiffany Edwards,<NA>,<NA>,<NA>
2,love_lies,"Love, Lies",43,<NA>,<NA>,<NA>,<NA>,<NA>,120,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",<NA>,<NA>,<NA>
3,the_sore_losers_1997,Sore Losers,60,<NA>,<NA>,<NA>,<NA>,2020-10-23,90,"Action, Mystery & thriller",English,John Michael McCarthy,John Michael McCarthy,<NA>,<NA>,<NA>
4,dinosaur_island_2002,Dinosaur Island,70,<NA>,<NA>,<NA>,<NA>,2017-03-27,80,"Fantasy, Adventure, Animation",English,Will Meugniot,John Loy,<NA>,<NA>,<NA>


Cleaning the data and adding some more informative columns that can be used to match entries between datasets.
- add releaseYear column from releaseDateTheaters and releaseDateStreaming, use the earlier one if both exists and do not match.
- remove entries that don't have both audienceScore and tomatoMeter

In [13]:
import datetime

In [48]:
test = np.where(df_rt['releaseDateStreaming'].fillna('nan').str.len() == 10, df_rt['releaseDateStreaming'],np.nan)

In [58]:
test2 = pd.to_datetime(test,format='ISO8601')

In [59]:
test_year = test2.year

In [66]:
test_year.fillna(-1).astype(int)

Index([2018, 2020,   -1, 2020, 2017, 2018, 2015,   -1,   -1, 2016,
       ...
         -1, 2016, 2013, 2017, 2019,   -1,   -1,   -1, 2006,   -1],
      dtype='int32', length=143258)

In [40]:
test = pd.to_datetime(df_rt['releaseDateStreaming'],format='ISO8601')

ValueError: Time data 1-01-01 is not ISO8601 format, at position 4446. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [33]:
df_rt[df_rt['releaseDateStreaming'] == "1-01-01"]

,id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix,releaseYearStreaming
101635,one_way_to_moscow,Moskau Einfach!,<NA>,<NA>,<NA>,<NA>,<NA>,1-01-01,93,Comedy,German,Micha Lewinsky,Micha Lewinsky,<NA>,<NA>,<NA>,2001.0


In [38]:
pd.Series(pd.to_datetime(["2020-01-01"])).dt.year

0    2020
dtype: int32

In [34]:
df_rt.loc[4446]

id                                  thalaikoothal
title                               Thalaikoothal
audienceScore                                <NA>
tomatoMeter                                  <NA>
rating                                       <NA>
ratingContents                               <NA>
releaseDateTheaters                          <NA>
releaseDateStreaming                         <NA>
runtimeMinutes                                160
genre                                       Drama
originalLanguage                            Tamil
director                Jayaprakash Radhakrishnan
writer                                       <NA>
boxOffice                                    <NA>
distributor                                  <NA>
soundMix                                     <NA>
releaseYearStreaming                          NaN
Name: 4446, dtype: object

In [19]:
df_rt['releaseYearStreaming'] = pd.to_datetime(df_rt['releaseDateStreaming'],format='mixed').dt.year

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143258 entries, 0 to 143257
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    143258 non-null  string 
 1   title                 142891 non-null  string 
 2   audienceScore         73248 non-null   Int64  
 3   tomatoMeter           33877 non-null   Int64  
 4   rating                13991 non-null   string 
 5   ratingContents        13991 non-null   string 
 6   releaseDateTheaters   30773 non-null   string 
 7   releaseDateStreaming  79420 non-null   string 
 8   runtimeMinutes        129431 non-null  Int64  
 9   genre                 132175 non-null  string 
 10  originalLanguage      129400 non-null  string 
 11  director              139041 non-null  string 
 12  writer                90116 non-null   string 
 13  boxOffice             14743 non-null   string 
 14  distributor           23001 non-null   string 
 15  

In [20]:
df_rt['releaseYearStreaming'] = pd.Series(df_rt['releaseYearStreaming'],dtype='int64')

In [27]:
import numpy as np
np.round(df_rt['releaseYearStreaming'])

0         2018.0
1         2020.0
2            NaN
3         2020.0
4         2017.0
           ...  
143253       NaN
143254       NaN
143255       NaN
143256    2006.0
143257       NaN
Name: releaseYearStreaming, Length: 143258, dtype: float64

In [21]:
df_rt.head()

,id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix,releaseYearStreaming
0,space-zombie-bingo,Space Zombie Bingo!,50,<NA>,<NA>,<NA>,<NA>,2018-08-25,75,"Comedy, Horror, Sci-fi",English,George Ormrod,"George Ormrod,John Sabotta",<NA>,<NA>,<NA>,2018.0
1,the_green_grass,The Green Grass,<NA>,<NA>,<NA>,<NA>,<NA>,2020-02-11,114,Drama,English,Tiffany Edwards,Tiffany Edwards,<NA>,<NA>,<NA>,2020.0
2,love_lies,"Love, Lies",43,<NA>,<NA>,<NA>,<NA>,<NA>,120,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",<NA>,<NA>,<NA>,NaN
3,the_sore_losers_1997,Sore Losers,60,<NA>,<NA>,<NA>,<NA>,2020-10-23,90,"Action, Mystery & thriller",English,John Michael McCarthy,John Michael McCarthy,<NA>,<NA>,<NA>,2020.0
4,dinosaur_island_2002,Dinosaur Island,70,<NA>,<NA>,<NA>,<NA>,2017-03-27,80,"Fantasy, Adventure, Animation",English,Will Meugniot,John Loy,<NA>,<NA>,<NA>,2017.0


In [5]:
# Change data types to correct types
df_basics['isAdult'] = df_basics['isAdult'].astype(bool)
df_basics['startYear']=pd.to_numeric(df_basics['startYear'], errors='coerce')
df_basics['endYear']=pd.to_numeric(df_basics['endYear'], errors='coerce')
df_basics['runtimeMinutes']=pd.to_numeric(df_basics['runtimeMinutes'], errors='coerce')
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11470477 entries, 0 to 11470476
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          string
 1   titleType       string
 2   primaryTitle    string
 3   originalTitle   string
 4   isAdult         bool  
 5   startYear       Int64 
 6   endYear         Int64 
 7   runtimeMinutes  Int64 
 8   genres          string
dtypes: Int64(3), bool(1), string(5)
memory usage: 743.9 MB


In [6]:
# Do the same with the ratings table
# Load TSV file into a DataFrame, specifying tab as the separator
df_ratings = pd.read_csv(os.path.join(path,'title.ratings.tsv'), sep='\t')
df_ratings = df_ratings.convert_dtypes()
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1537005 entries, 0 to 1537004
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1537005 non-null  string 
 1   averageRating  1537005 non-null  Float64
 2   numVotes       1537005 non-null  Int64  
dtypes: Float64(1), Int64(1), string(1)
memory usage: 38.1 MB


In [7]:
# Connect to SQLite database 
conn = sqlite3.connect('films.db') 

# Create a cursor object 
cur = conn.cursor() 

# Write the data to a sqlite table 
df_basics.to_sql('imdb_basics', conn, if_exists='replace', index=False) 
df_ratings.to_sql('imdb_ratings', conn, if_exists='replace', index=False) 

1537005

In [8]:
# Check if basics dataframe was saved correctly by loading and pulling up first 10 entries
query = """
        SELECT *
        FROM imdb_basics
        LIMIT 10;
        """
for row in cur.execute(query): 
    print(row) 

('tt0000001', 'short', 'Carmencita', 'Carmencita', 0, 1894, None, 1, 'Documentary,Short')
('tt0000002', 'short', 'Le clown et ses chiens', 'Le clown et ses chiens', 0, 1892, None, 5, 'Animation,Short')
('tt0000003', 'short', 'Poor Pierrot', 'Pauvre Pierrot', 0, 1892, None, 5, 'Animation,Comedy,Romance')
('tt0000004', 'short', 'Un bon bock', 'Un bon bock', 0, 1892, None, 12, 'Animation,Short')
('tt0000005', 'short', 'Blacksmith Scene', 'Blacksmith Scene', 0, 1893, None, 1, 'Short')
('tt0000006', 'short', 'Chinese Opium Den', 'Chinese Opium Den', 0, 1894, None, 1, 'Short')
('tt0000007', 'short', 'Corbett and Courtney Before the Kinetograph', 'Corbett and Courtney Before the Kinetograph', 0, 1894, None, 1, 'Short,Sport')
('tt0000008', 'short', 'Edison Kinetoscopic Record of a Sneeze', 'Edison Kinetoscopic Record of a Sneeze', 0, 1894, None, 1, 'Documentary,Short')
('tt0000009', 'movie', 'Miss Jerry', 'Miss Jerry', 0, 1894, None, 45, 'Romance')
('tt0000010', 'short', 'Leaving the Factory',

In [9]:
# Check if ratings dataframe was saved correctly by loading and pulling up first 10 entries
query = """
        SELECT *
        FROM imdb_ratings
        LIMIT 10;
        """
for row in cur.execute(query): 
    print(row) 

('tt0000001', 5.7, 2133)
('tt0000002', 5.5, 289)
('tt0000003', 6.4, 2169)
('tt0000004', 5.3, 184)
('tt0000005', 6.2, 2896)
('tt0000006', 5.0, 208)
('tt0000007', 5.3, 901)
('tt0000008', 5.4, 2280)
('tt0000009', 5.3, 220)
('tt0000010', 6.8, 7871)


### Let's load Netflix original film list and add to the database

First, I want to add the netflix data to the database so we can access everything from one place in the future.

In [10]:
# Load TSV file into a DataFrame, specifying tab as the separator
df_netflix = pd.read_csv('netflix_og_films.csv')
# Fix the column names before writing to the database to follow SQL syntax
df_netflix.rename(columns={'Title': 'title', 'Release date': 'releaseDate', 
                   'Genre':'genre','Runtime':'runtime','Language':'language','Film type':'filmType'}, inplace=True)

Before we save this dataframe into the database, I want to convert runtime ('x h x min') to minutes, so that we can use this as an additional constraint when matching the title of the films to IMDB.

In [11]:
def hm2m(time_str):
    """a function to convert runtime to minutes"""
    cuml_min = 0
    if time_str.find('h')>0:
        h_pos = time_str.find('h')
        cuml_min += int(time_str[:h_pos-1])*60
        time_str = time_str[h_pos+2:]
    
    if time_str.find('min')>0:
        m_pos = time_str.find('min')
        cuml_min += int(time_str[:m_pos-1])
        
    return cuml_min

In [12]:
# Convert runtime that's in X h X min to minutes
df_netflix = df_netflix.convert_dtypes() # convert to string first
df_netflix['runtime_min']=df_netflix['runtime'].apply(hm2m) # apply the above function to the runtime column
df_netflix.head() # check if it worked correctly

,title,releaseDate,genre,runtime,language,filmType,runtime_min
0,Beasts of No Nation,"October 16, 2015",War drama,2 h 17 min,English,Feature films,137
1,The Ridiculous 6,"December 11, 2015",Western comedy,2 h,English,Feature films,120
2,Pee-wee's Big Holiday,"March 18, 2016",Adventure comedy,1 h 30 min,English,Feature films,90
3,Special Correspondents,"April 29, 2016",Satire,1 h 41 min,English,Feature films,101
4,The Do-Over,"May 27, 2016",Action comedy,1 h 48 min,English,Feature films,108


In [28]:
df_netflix['releaseYear'] = df_netflix['releaseDate'].str[-4:].astype(int)

In [29]:
df_netflix.head()

,title,releaseDate,genre,runtime,language,filmType,runtime_min,releaseYear
0,Beasts of No Nation,"October 16, 2015",War drama,2 h 17 min,English,Feature films,137,2015
1,The Ridiculous 6,"December 11, 2015",Western comedy,2 h,English,Feature films,120,2015
2,Pee-wee's Big Holiday,"March 18, 2016",Adventure comedy,1 h 30 min,English,Feature films,90,2016
3,Special Correspondents,"April 29, 2016",Satire,1 h 41 min,English,Feature films,101,2016
4,The Do-Over,"May 27, 2016",Action comedy,1 h 48 min,English,Feature films,108,2016


In [30]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1413 non-null   string
 1   releaseDate  1413 non-null   string
 2   genre        1065 non-null   string
 3   runtime      1413 non-null   string
 4   language     1413 non-null   string
 5   filmType     1413 non-null   string
 6   runtime_min  1413 non-null   int64 
 7   releaseYear  1413 non-null   int64 
dtypes: int64(2), string(6)
memory usage: 88.4 KB


In [32]:
# Write the data to a sqlite table 
df_netflix.to_sql('netflix', conn, if_exists='replace', index=False) 

1413

In [33]:
# Read in data and list the column names and the first 10 entries
query = """
        SELECT *
        FROM netflix
        LIMIT 10;
        """
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 

['title', 'releaseDate', 'genre', 'runtime', 'language', 'filmType', 'runtime_min', 'releaseYear']
('Beasts of No Nation', 'October 16, 2015', 'War drama', '2 h 17 min', 'English', 'Feature films', 137, 2015)
('The Ridiculous 6', 'December 11, 2015', 'Western comedy', '2 h', 'English', 'Feature films', 120, 2015)
("Pee-wee's Big Holiday", 'March 18, 2016', 'Adventure comedy', '1 h 30 min', 'English', 'Feature films', 90, 2016)
('Special Correspondents', 'April 29, 2016', 'Satire', '1 h 41 min', 'English', 'Feature films', 101, 2016)
('The Do-Over', 'May 27, 2016', 'Action comedy', '1 h 48 min', 'English', 'Feature films', 108, 2016)
('The Fundamentals of Caring', 'June 24, 2016', 'Comedy drama', '1 h 37 min', 'English', 'Feature films', 97, 2016)
('Brahman Naman', 'July 7, 2016', 'Sex comedy', '1 h 35 min', 'English', 'Feature films', 95, 2016)
('Rebirth', 'July 15, 2016', 'Thriller', '1 h 40 min', 'English', 'Feature films', 100, 2016)
('Tallulah', 'July 29, 2016', 'Comedy drama', '1 

Close out the connection to SQLite database

In [34]:
# Close connection to SQLite database 
conn.close() 